In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

from tqdm import tqdm
import numpy as np


python_path = "/home/yskim/projects/sparse-bfp"
import sys
if python_path not in sys.path:
    sys.path.append(python_path)

from util.reprod_util import set_reproducibility

from util.custom_linear import CustomLinear, PrecisionFlag
from util.custom_conv2d import CustomConv2d

set_reproducibility(random_seed=128)

from util.data_load import *

from util.bfp.bfp_config import BfpConfig
BfpConfig.bfp_M_Bit = 8

from util.custom_linear import LinearLogConfig
from util.custom_conv2d import Conv2dLogConfig

In [2]:
batch_size = 32
# train_loader, valid_loader = generate_data_loaders(type=DL_TYPE_IMAGENET, batch_size=32)
train_loader, valid_loader = generate_data_loaders(type=DL_TYPE_CIFAR_100, batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
def wrap_modules(module, precision_flag: PrecisionFlag):
    for name, _ in module.named_children():
        child = getattr(module, name)
        child_cnt = 0
        for _ in child.children():
            child_cnt += 1

        if child_cnt == 0:
            layer = getattr(module, name)
            for _, param in layer.named_parameters():
                if param.requires_grad:
                    pass
                    # print(param.shape)
            # print()
            if "Linear" in str(layer):
                custom_layer = CustomLinear(input_features=layer.in_features, 
                                            output_features=layer.out_features, 
                                            bias=True if layer.bias is not None else False, 
                                            precision_flag=precision_flag)
                custom_layer.weight = nn.Parameter(layer.weight.data.clone().detach())
                if layer.bias is not None:
                    custom_layer.bias = nn.Parameter(layer.bias.data.clone().detach())
                setattr(module, name, custom_layer)
            elif "Conv" in str(layer):
                custom_layer = CustomConv2d(in_channels=layer.in_channels,
                                            out_channels=layer.out_channels,
                                            kernel_size=layer.kernel_size[0],
                                            stride=layer.stride[0],
                                            padding=layer.padding[0],
                                            bias=True if layer.bias is not None else False,
                                            precision_flag=precision_flag)
                custom_layer.weight = nn.Parameter(layer.weight.data.clone().detach())
                if layer.bias is not None:
                    custom_layer.bias = nn.Parameter(layer.bias.data.clone().detach())
                setattr(module, name, custom_layer)
        else:
            wrap_modules(child, precision_flag)


def change_to_leaky_relu(module):
    for name, _ in module.named_children():
        child = getattr(module, name)
        child_cnt = 0
        for _ in child.children():
            child_cnt += 1

        if child_cnt == 0:
            layer = getattr(module, name)
            for _, param in layer.named_parameters():
                if param.requires_grad:
                    pass
                    # print(param.shape)
            # print()
            if "ReLU" in str(layer):
                leaky_relu = nn.LeakyReLU()
                setattr(module, name, leaky_relu)
                print("changed!")
        else:
            change_to_leaky_relu(child)

In [4]:
# device = 'cpu'
# BfpConfig.bfp_M_Bit = 8

# model_ref = models.vgg13_bn(num_classes=100)
# # change_to_leaky_relu(model_ref)

# name = "tv-vgg13_bn-b32-fp-cuda"
# epoch = 30

# path = f"/home/yskim/projects/sparse-bfp/train-log/{name}/epoch-{epoch}.pth"
# model_ref.load_state_dict(torch.load(path)["model_state_dict"])

# criterion = torch.nn.CrossEntropyLoss()
# optimizer = optim.Adam(model_ref.parameters(), lr=0.01, weight_decay=1e-4)
# # optimizer = optim.SGD(model_ref.parameters(), lr=0.01)


In [5]:
device = 'cpu'
BfpConfig.bfp_M_Bit = 2
CustomConv2d.current_id = 0
CustomLinear.current_id = 0

model_ref = models.vgg13_bn(num_classes=100)
# change_to_leaky_relu(model_ref)

name = "tv-vgg13_bn-b32-fp-cuda"
epoch = 30

path = f"/home/yskim/projects/sparse-bfp/train-log/{name}/epoch-{epoch}.pth"
model_ref.load_state_dict(torch.load(path)["model_state_dict"])

lr = 0.01

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ref.parameters(), lr=lr, weight_decay=1e-4)
# optimizer = optim.SGD(model_ref.parameters(), lr=0.01)

precision_flag = PrecisionFlag.BFP
wrap_modules(model_ref, precision_flag=precision_flag)

LinearLogConfig.should_log = True
Conv2dLogConfig.should_log = True

model_ref = model_ref.to(device)
criterion = criterion.to(device)

log_root = "/home/yskim/projects/sparse-bfp/bfp-iter-log"
data_format = f"bfp-m{BfpConfig.bfp_M_Bit}" if precision_flag == PrecisionFlag.BFP else "fp"

path_root = f"{log_root}/{name}-lr-{lr}-e{epoch}"

if os.path.isdir(path_root):
    print(f"already have a directory: {path_root}")
    # raise ValueError(f"already have a directory: {log_root}/{name}")
else:
    os.mkdir(f"{path_root}")

os.mkdir(f"{path_root}/{data_format}")

path = f"{path_root}/{data_format}"

model_ref = model_ref.to(device)
criterion = criterion.to(device)


best_valid_acc = float("-inf")

iterations = 1
curr_iter = 0
for epoch in range(1):
    model_ref.train()

    for batch_index, (inputs, targets) in enumerate(tqdm(train_loader), 0):
        if curr_iter >= iterations:
            break

        curr_path = f"{path}/iter-{curr_iter}"
        os.mkdir(curr_path)
        Conv2dLogConfig.path = curr_path
        LinearLogConfig.path = curr_path

        train_top1_correct_cnt = 0
        train_top5_correct_cnt = 0
        train_total_cnt = 0
        train_loss = 0

        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model_ref(inputs)

        pred_labels = torch.argmax(outputs, axis=1)
        
        train_top1_correct_cnt += torch.sum(pred_labels == targets).item()

        _, indices = torch.topk(outputs, k=5)
        for b in range(outputs.shape[0]):
            if targets[b] in indices[b]:
                train_top5_correct_cnt += 1
                continue

        train_total_cnt += outputs.shape[0]

        loss = criterion(outputs, targets)

        train_loss += loss.item() * outputs.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss /= train_total_cnt
        train_top1_acc = train_top1_correct_cnt / train_total_cnt * 100.0
        train_top5_acc = train_top5_correct_cnt / train_total_cnt * 100.0

        print(f"[{curr_iter}th iter train accuracy] "
              f"top1: {train_top1_acc:.2f}%, "
              f"top5: {train_top5_acc:.2f}%, "
              f"loss: {train_loss}")

        torch.save({
            'train_loss': train_loss,
            'train_top1_accuracy': train_top1_acc,
            'train_top5_accuracy': train_top5_acc,
        }, f"{curr_path}/iter_info.pth")

        curr_iter += 1

  0%|          | 1/1563 [00:15<6:52:19, 15.84s/it]

[0th iter train accuracy] top1: 40.62%, top5: 62.50%, loss: 3.3153488636016846


In [6]:
device = 'cpu'
BfpConfig.bfp_M_Bit = 4
CustomConv2d.current_id = 0
CustomLinear.current_id = 0

model_ref = models.vgg13_bn(num_classes=100)
# change_to_leaky_relu(model_ref)

name = "tv-vgg13_bn-b32-fp-cuda"
epoch = 30

path = f"/home/yskim/projects/sparse-bfp/train-log/{name}/epoch-{epoch}.pth"
model_ref.load_state_dict(torch.load(path)["model_state_dict"])

lr = 0.01

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ref.parameters(), lr=lr, weight_decay=1e-4)
# optimizer = optim.SGD(model_ref.parameters(), lr=0.01)

precision_flag = PrecisionFlag.BFP
wrap_modules(model_ref, precision_flag=precision_flag)

LinearLogConfig.should_log = True
Conv2dLogConfig.should_log = True

model_ref = model_ref.to(device)
criterion = criterion.to(device)

log_root = "/home/yskim/projects/sparse-bfp/bfp-iter-log"
data_format = f"bfp-m{BfpConfig.bfp_M_Bit}" if precision_flag == PrecisionFlag.BFP else "fp"

path_root = f"{log_root}/{name}-lr-{lr}-e{epoch}"

if os.path.isdir(path_root):
    print(f"already have a directory: {path_root}")
    # raise ValueError(f"already have a directory: {log_root}/{name}")
else:
    os.mkdir(f"{path_root}")

os.mkdir(f"{path_root}/{data_format}")

path = f"{path_root}/{data_format}"

model_ref = model_ref.to(device)
criterion = criterion.to(device)


best_valid_acc = float("-inf")

iterations = 1
curr_iter = 0
for epoch in range(1):
    model_ref.train()

    for batch_index, (inputs, targets) in enumerate(tqdm(train_loader), 0):
        if curr_iter >= iterations:
            break

        curr_path = f"{path}/iter-{curr_iter}"
        os.mkdir(curr_path)
        Conv2dLogConfig.path = curr_path
        LinearLogConfig.path = curr_path

        train_top1_correct_cnt = 0
        train_top5_correct_cnt = 0
        train_total_cnt = 0
        train_loss = 0

        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model_ref(inputs)

        pred_labels = torch.argmax(outputs, axis=1)
        
        train_top1_correct_cnt += torch.sum(pred_labels == targets).item()

        _, indices = torch.topk(outputs, k=5)
        for b in range(outputs.shape[0]):
            if targets[b] in indices[b]:
                train_top5_correct_cnt += 1
                continue

        train_total_cnt += outputs.shape[0]

        loss = criterion(outputs, targets)

        train_loss += loss.item() * outputs.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss /= train_total_cnt
        train_top1_acc = train_top1_correct_cnt / train_total_cnt * 100.0
        train_top5_acc = train_top5_correct_cnt / train_total_cnt * 100.0

        print(f"[{curr_iter}th iter train accuracy] "
              f"top1: {train_top1_acc:.2f}%, "
              f"top5: {train_top5_acc:.2f}%, "
              f"loss: {train_loss}")

        torch.save({
            'train_loss': train_loss,
            'train_top1_accuracy': train_top1_acc,
            'train_top5_accuracy': train_top5_acc,
        }, f"{curr_path}/iter_info.pth")

        curr_iter += 1

already have a directory: /home/yskim/projects/sparse-bfp/bfp-iter-log/tv-vgg13_bn-b32-fp-cuda-lr-0.01-e30


  0%|          | 1/1563 [00:15<6:44:12, 15.53s/it]

[0th iter train accuracy] top1: 93.75%, top5: 100.00%, loss: 0.3070448637008667


In [7]:
device = 'cpu'
BfpConfig.bfp_M_Bit = 6
CustomConv2d.current_id = 0
CustomLinear.current_id = 0

model_ref = models.vgg13_bn(num_classes=100)
# change_to_leaky_relu(model_ref)

name = "tv-vgg13_bn-b32-fp-cuda"
epoch = 30

path = f"/home/yskim/projects/sparse-bfp/train-log/{name}/epoch-{epoch}.pth"
model_ref.load_state_dict(torch.load(path)["model_state_dict"])

lr = 0.01

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ref.parameters(), lr=lr, weight_decay=1e-4)
# optimizer = optim.SGD(model_ref.parameters(), lr=0.01)

precision_flag = PrecisionFlag.BFP
wrap_modules(model_ref, precision_flag=precision_flag)

LinearLogConfig.should_log = True
Conv2dLogConfig.should_log = True

model_ref = model_ref.to(device)
criterion = criterion.to(device)

log_root = "/home/yskim/projects/sparse-bfp/bfp-iter-log"
data_format = f"bfp-m{BfpConfig.bfp_M_Bit}" if precision_flag == PrecisionFlag.BFP else "fp"

path_root = f"{log_root}/{name}-lr-{lr}-e{epoch}"

if os.path.isdir(path_root):
    print(f"already have a directory: {path_root}")
    # raise ValueError(f"already have a directory: {log_root}/{name}")
else:
    os.mkdir(f"{path_root}")

os.mkdir(f"{path_root}/{data_format}")

path = f"{path_root}/{data_format}"

model_ref = model_ref.to(device)
criterion = criterion.to(device)


best_valid_acc = float("-inf")

iterations = 1
curr_iter = 0
for epoch in range(1):
    model_ref.train()

    for batch_index, (inputs, targets) in enumerate(tqdm(train_loader), 0):
        if curr_iter >= iterations:
            break

        curr_path = f"{path}/iter-{curr_iter}"
        os.mkdir(curr_path)
        Conv2dLogConfig.path = curr_path
        LinearLogConfig.path = curr_path

        train_top1_correct_cnt = 0
        train_top5_correct_cnt = 0
        train_total_cnt = 0
        train_loss = 0

        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model_ref(inputs)

        pred_labels = torch.argmax(outputs, axis=1)
        
        train_top1_correct_cnt += torch.sum(pred_labels == targets).item()

        _, indices = torch.topk(outputs, k=5)
        for b in range(outputs.shape[0]):
            if targets[b] in indices[b]:
                train_top5_correct_cnt += 1
                continue

        train_total_cnt += outputs.shape[0]

        loss = criterion(outputs, targets)

        train_loss += loss.item() * outputs.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss /= train_total_cnt
        train_top1_acc = train_top1_correct_cnt / train_total_cnt * 100.0
        train_top5_acc = train_top5_correct_cnt / train_total_cnt * 100.0

        print(f"[{curr_iter}th iter train accuracy] "
              f"top1: {train_top1_acc:.2f}%, "
              f"top5: {train_top5_acc:.2f}%, "
              f"loss: {train_loss}")

        torch.save({
            'train_loss': train_loss,
            'train_top1_accuracy': train_top1_acc,
            'train_top5_accuracy': train_top5_acc,
        }, f"{curr_path}/iter_info.pth")

        curr_iter += 1

already have a directory: /home/yskim/projects/sparse-bfp/bfp-iter-log/tv-vgg13_bn-b32-fp-cuda-lr-0.01-e30


  0%|          | 1/1563 [00:14<6:28:58, 14.94s/it]

[0th iter train accuracy] top1: 96.88%, top5: 100.00%, loss: 0.07790787518024445


In [8]:
device = 'cpu'
CustomConv2d.current_id = 0
CustomLinear.current_id = 0

model_ref = models.vgg13_bn(num_classes=100)
# change_to_leaky_relu(model_ref)

name = "tv-vgg13_bn-b32-fp-cuda"
epoch = 30

path = f"/home/yskim/projects/sparse-bfp/train-log/{name}/epoch-{epoch}.pth"
model_ref.load_state_dict(torch.load(path)["model_state_dict"])

lr = 0.01

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ref.parameters(), lr=lr, weight_decay=1e-4)
# optimizer = optim.SGD(model_ref.parameters(), lr=0.01)

precision_flag = PrecisionFlag.FP
wrap_modules(model_ref, precision_flag=precision_flag)

LinearLogConfig.should_log = True
Conv2dLogConfig.should_log = True

model_ref = model_ref.to(device)
criterion = criterion.to(device)

log_root = "/home/yskim/projects/sparse-bfp/bfp-iter-log"
data_format = f"bfp-m{BfpConfig.bfp_M_Bit}" if precision_flag == PrecisionFlag.BFP else "fp"
# data_format = "fp"

path_root = f"{log_root}/{name}-lr-{lr}-e{epoch}"

if os.path.isdir(f"{path_root}"):
    print(f"already have a directory: {path_root}")
    # raise ValueError(f"already have a directory: {log_root}/{name}-e{epoch}")
else:
    os.mkdir(f"{path_root}")
    
os.mkdir(f"{path_root}/{data_format}")

# os.mkdir(f"{log_root}/{name}/{data_format}")

path = f"{path_root}/{data_format}"

model_ref = model_ref.to(device)
criterion = criterion.to(device)


best_valid_acc = float("-inf")

iterations = 1
curr_iter = 0
for epoch in range(1):
    model_ref.train()

    for batch_index, (inputs, targets) in enumerate(tqdm(train_loader), 0):
        if curr_iter >= iterations:
            break

        curr_path = f"{path}/iter-{curr_iter}"
        os.mkdir(curr_path)
        Conv2dLogConfig.path = curr_path
        LinearLogConfig.path = curr_path

        train_top1_correct_cnt = 0
        train_top5_correct_cnt = 0
        train_total_cnt = 0
        train_loss = 0

        inputs = inputs.to(device)
        targets = targets.to(device)

        outputs = model_ref(inputs)

        pred_labels = torch.argmax(outputs, axis=1)
        
        train_top1_correct_cnt += torch.sum(pred_labels == targets).item()

        _, indices = torch.topk(outputs, k=5)
        for b in range(outputs.shape[0]):
            if targets[b] in indices[b]:
                train_top5_correct_cnt += 1
                continue

        train_total_cnt += outputs.shape[0]

        loss = criterion(outputs, targets)

        train_loss += loss.item() * outputs.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss /= train_total_cnt
        train_top1_acc = train_top1_correct_cnt / train_total_cnt * 100.0
        train_top5_acc = train_top5_correct_cnt / train_total_cnt * 100.0

        print(f"[{curr_iter}th iter train accuracy] "
              f"top1: {train_top1_acc:.2f}%, "
              f"top5: {train_top5_acc:.2f}%, "
              f"loss: {train_loss}")

        torch.save({
            'train_loss': train_loss,
            'train_top1_accuracy': train_top1_acc,
            'train_top5_accuracy': train_top5_acc,
        }, f"{curr_path}/iter_info.pth")

        curr_iter += 1

already have a directory: /home/yskim/projects/sparse-bfp/bfp-iter-log/tv-vgg13_bn-b32-fp-cuda-lr-0.01-e30


  0%|          | 1/1563 [00:04<2:03:53,  4.76s/it]

[0th iter train accuracy] top1: 100.00%, top5: 100.00%, loss: 0.039249077439308167
